In [3]:
using DataStructures
using Distributions
using StableRNGs
using Printf
using Dates

### use one global variable
const n_servers = 2

### Entity data structure for each order
### Modified Order structure to include queue waiting times
mutable struct Order
    id::Int64
    arrival_time::Float64
    start_service_times::Array{Float64,1}
    completion_time::Float64
    queue_wait_times::Array{Float64,1}  # New: time spent waiting in each queue
    total_time_in_system::Float64  # New: total time spent in the system
end
Order(id::Int64, arrival_time::Float64 ) = Order(id, arrival_time, Array{Float64,1}(undef,2), Inf, zeros(Float64, 2), 0.0)

### Events
abstract type Event end 

struct Arrival <: Event # order arrives
    id::Int64         # a unique event id
    time::Float64     # the time of the event 
end

mutable struct Finish <: Event # an order finishes processing at machine i
    id::Int64         # a unique event id
    time::Float64     # the time of the event
    server::Int64     # ID of the server that is finishing
end

struct Null <: Event # order arrives
    id::Int64    
end

### parameter structure
struct Parameters
    seed::Int
    T::Float64
    mean_interarrival::Float64
    mean_machine_times::Array{Float64,1}
    max_queue::Array{Int64,1}          # space available in each queue
    time_units::String
end
function write_parameters( output::IO, P::Parameters ) # function to writeout parameters
    T = typeof(P)
    for name in fieldnames(T)
        println( output, "# parameter: $name = $(getfield(P,name))" )
    end
end
write_parameters( P::Parameters ) = write_parameters( stdout, P )
function write_metadata( output::IO ) # function to writeout extra metadata
    (path, prog) = splitdir( @__FILE__ )
    println( output, "# file created by code in $(prog)" )
    t = now()
    println( output, "# file created on $(Dates.format(t, "yyyy-mm-dd at HH:MM:SS"))" )
end

### State
mutable struct SystemState
    time::Float64                               # the system time (simulation time)
    n_entities::Int64                           # the number of entities to have been served
    n_events::Int64                             # tracks the number of events to have occur + queued
    event_queue::PriorityQueue{Event,Float64}   # to keep track of future arravals/services
    order_queues::Array{Queue{Order},1}         # the system queues (1 is the arrival queue)
    in_service::Array{Union{Order,Nothing},1}   # the order currently in service at machine i if there is one
    machine_idle_times::Array{Float64,1}  # New: total idle time for each machine
end
function SystemState( P::Parameters ) # create an initial (empty) state
    init_time = 0.0
    init_n_entities = 0
    init_n_events = 0
    init_event_queue = PriorityQueue{Event,Float64}()
    init_order_queues = Array{Queue{Order},1}(undef,n_servers)
    machine_idle_times= zeros(Float64, n_servers)
    for i=1:n_servers
        init_order_queues[i] = Queue{Order}()
    end
    init_in_service = Array{Union{Order,Nothing},1}(undef,n_servers)
    for i=1:n_servers
        init_in_service[i] = nothing
    end
    return SystemState( init_time,
                        init_n_entities,
                        init_n_events,
                        init_event_queue,
                        init_order_queues,
                        init_in_service,
                        machine_idle_times)
end

# setup random number generators
struct RandomNGs
    rng::StableRNGs.LehmerRNG
    interarrival_time::Function
    machine_times::Array{Function,1}
end
# constructor function to create all the pieces required
function RandomNGs( P::Parameters )
    rng = StableRNG( P.seed ) # create a new RNG with seed set to that required
    interarrival_time() = rand(rng, Exponential(P.mean_interarrival))  

    machine_times = Array{Function,1}(undef,n_servers)
    machine_times[1] = () -> rand(rng, Normal(mean_machine_times[1], sd_machine_times[1]))                # create this as a function to be consistent
    machine_times[2] = () -> rand(rng, Exponential(P.mean_machine_times[2]))

    return RandomNGs( rng, interarrival_time,  machine_times )
end

# initialisation function for the simulation
function initialise( P::Parameters )
    # construct random number generators and system state
    R = RandomNGs( P )
    system = SystemState( P )

    # add an arrival at time 0.0
    t0 = 0.0
    system.n_events += 1
    enqueue!( system.event_queue, Arrival(0,t0),t0)

    return (system, R)
end

### output functions (I am using formatted output, but it could use just println)
function write_state(event_file::IO, system::SystemState, event::Event, timing::AbstractString; debug_level::Int=0)
    if typeof(event) <: Finish
        type_of_event = "Finish($(event.server))"
    else
        type_of_event = typeof(event)
    end

    # Adding idle time for each machine to the formatted output.
    @printf(event_file,
            "%12.3f,%6d,%9s,%6s,%4d,%4d,%4d,%4d,%4d, %12.3f, %12.3f\n",
            system.time,
            event.id,
            type_of_event,
            timing,
            length(system.event_queue),
            length(system.order_queues[1]),
            length(system.order_queues[2]),
            system.in_service[1] == nothing ? 0 : 1,
            system.in_service[2] == nothing ? 0 : 1,
            system.machine_idle_times[1],  # Idle time for machine 1
            system.machine_idle_times[2]   # Idle time for machine 2
           )
end

function write_entity_header( entity_file::IO, entity )
    T = typeof( entity )
    x = Array{Any,1}(undef, length( fieldnames(typeof(entity)) ) )
    for (i,name) in enumerate(fieldnames(T))
        tmp = getfield(entity,name)
        if isa(tmp, Array)
            x[i] = join( repeat( [name], length(tmp) ), ',' )
        else
            x[i] = name
        end
    end
    println( entity_file, join( x, ',') )
end
function write_entity( entity_file::IO, entity; debug_level::Int=0)
    T = typeof( entity )
    x = Array{Any,1}(undef,length( fieldnames(typeof(entity)) ) )
    for (i,name) in enumerate(fieldnames(T))
        tmp = getfield(entity,name)
        if isa(tmp, Array)
            x[i] = join( tmp, ',' )
        else
            x[i] = tmp
        end
    end
    println( entity_file, join( x, ',') )
end

using Statistics

# Function to calculate confidence interval
function confidence_interval(data, confidence_level=0.95)
    mean_val = mean(data)
    std_err = std(data) / sqrt(length(data))
    z = quantile(Normal(0, 1), 1 - (1 - confidence_level) / 2)
    lower_bound = mean_val - z * std_err
    upper_bound = mean_val + z * std_err
    return (lower_bound, upper_bound)
end

### Update functions
function update!( system::SystemState, P::Parameters, R::RandomNGs, e::Event )
    throw( DomainError("invalid event type" ) )
end

function move_to_server!( system::SystemState, R::RandomNGs, server::Integer )
    # move the order order from a queue into construction
    system.in_service[server] = dequeue!(system.order_queues[server]) 
    system.in_service[server].start_service_times[server] = system.time # start service 'now'
    completion_time = system.time + R.machine_times[server]() # best current guess at service time
    
    # New: update queue wait time
    order = system.in_service[server]
    order.queue_wait_times[server] = system.time - order.arrival_time
    
    # create a finish event for the machine current constructing the item
    system.n_events += 1
    finish_event = Finish( system.n_events, completion_time, server )
    enqueue!( system.event_queue, finish_event, completion_time )
    return nothing
end

function update!( system::SystemState, P::Parameters, R::RandomNGs, event::Arrival )
    # create an arriving order and add it to the 1st queue
    system.n_entities += 1    # new entity will enter the system
    new_order = Order( system.n_entities, event.time )
    enqueue!(system.order_queues[1], new_order)
    
    # generate next arrival and add it to the event queue
    future_arrival = Arrival(system.n_events, system.time + R.interarrival_time())
    enqueue!(system.event_queue, future_arrival, future_arrival.time)

    # if the construction machine is available, the order goes to service
    if system.in_service[1] == nothing
        move_to_server!( system, R, 1 )
    end
    return nothing
end

function stall_event!( system::SystemState, event::Event )
    # defer an event until after the next event in the list
    next_event_time = peek( system.event_queue )[2]
    event.time = next_event_time + eps() # add eps() so that this event occurs just after the next event
    enqueue!(system.event_queue, event, event.time)
    return nothing
end

function update!( system::SystemState, P::Parameters, R::RandomNGs, event::Finish )
    server = event.server
    if server < n_servers && length(system.order_queues[server+1]) >= P.max_queue[server+1]
        # if the server finishes, but there are too many people in the next queue,
        # then defer the event until the queue has space, i.e, the next finish event
        # but finding the next event is easy, and next finish is hard, so we stall by one
        stall_event!( system, event )
    else
        # otherwise treat this as normal finish of service
        departing_order = deepcopy( system.in_service[server] )
        system.in_service[server] = nothing
        
        if !isempty(system.order_queues[server]) # if someone is waiting, move them to service
            move_to_server!( system, R, server )
            
            # New: update idle time, assuming next event immediately follows
            next_event_time = isempty(system.event_queue) ? P.T : peek(system.event_queue)[2]
            system.machine_idle_times[server] += next_event_time - system.time
        end

        if server < n_servers
            # move the customer to the next queue
            enqueue!(system.order_queues[server+1], departing_order)
            if system.in_service[server+1] === nothing
                move_to_server!( system, R, server+1 )
            end
        else
            # or return the entity when it is leaving the system for good
            departing_order.completion_time = system.time
            departing_order.total_time_in_system = system.time - departing_order.arrival_time  # Calculate total time in system
            return departing_order 
        end
    end
    return nothing
end

function run!( system::SystemState, P::Parameters, R::RandomNGs, fid_state::IO, fid_entities::IO; output_level::Integer=2)
    # main simulation loop
    while system.time < P.T
        if P.seed ==1 && system.time <= 1000.0
            println("$(system.time): ") # debug information for first few events whenb seed = 1
        end

        # grab the next event from the event queue
        (event, time) = dequeue_pair!(system.event_queue)
        system.time = time  # advance system time to the new arrival
        system.n_events += 1      # increase the event counter
        
        # write out event and state data before event
        if output_level>=2
            write_state( fid_state, system, event, "before")
        elseif output_level==1 && typeof(event) == Arrival
            write_state( fid_state, system, event, "before")
        end
        
        # update the system based on the next event, and spawn new events. 
        # return arrived/departed customer.
        departure = update!( system, P, R, event )
         
        # write out event and state data after event for debugging
        if output_level>=2
            write_state( fid_state, system, event, "after")
        end
        
        # write out entity data if it was a departure from the system
        if departure !== nothing && output_level>=2
            write_entity( fid_entities, departure )
        end
    end
    return system
end



# inititialise
seed = 1
T = 10_000.0
mean_interarrival = 60.0 ###    units here are minutes
mean_machine_times = [25.0, 59.0]
sd_machine_times = [7.18] 
max_queue = [typemax(Int64), 4]
time_units = "minutes"
P = Parameters( seed, T, mean_interarrival, mean_machine_times, max_queue, time_units)

# file directory and name; * concatenates strings.
dir = pwd()*"/data/"*"/seed"*string(P.seed) # directory name
mkpath(dir)                          # this creates the directory 
file_entities = dir*"/entities.csv"  # the name of the data file (informative) 
file_state = dir*"/state.csv"        # the name of the data file (informative) 
fid_entities = open(file_entities, "w") # open the file for writing
fid_state = open(file_state, "w")       # open the file for writing

write_metadata( fid_entities )
write_metadata( fid_state )
write_parameters( fid_entities, P )
write_parameters( fid_state, P )

# headers
write_entity_header( fid_entities,  Order(0, 0.0) )
println(fid_state,"time,event_id,event_type,timing,length_event_list,length_queue1,length_queue2,in_service1,in_service2,idlet_m1,idlet_m2")

# run the actual simulation
(system,R) = initialise( P ) 
run!( system, P, R, fid_state, fid_entities)

# remember to close the files
close( fid_entities )
close( fid_state )


# Function to run a single simulation and return collected statistics
# Function to run a single simulation and return collected statistics
function run_single_simulation(seed::Int)
    P = Parameters(seed, 10_000.0, 60.0, [25.0, 59.0], [typemax(Int64), 4], "minutes")
    (system, R) = initialise(P)

    n_entities_in_system_over_time = []  # Track number of entities in the system
    queue1_wait_times = []
    queue2_wait_times = []
    total_times_in_system = []
    finished_entities_count = 0  # Counter for finished entities
    queue_lengths_over_time = []  # Store queue lengths for each time step
    interarrival_times = []
    last_arrival_time = 0.0
    
    while system.time < P.T
        (event, time) = dequeue_pair!(system.event_queue)
        system.time = time
        departure = update!(system, P, R, event)
        if typeof(event) == Arrival
            push!(interarrival_times, event.time - last_arrival_time)
            last_arrival_time = event.time
        end
        if typeof(departure) == Order
            push!(queue1_wait_times, departure.queue_wait_times[1])
            push!(queue2_wait_times, departure.queue_wait_times[2])
            push!(total_times_in_system, departure.total_time_in_system)
            push!(queue_lengths_over_time, length(system.order_queues[1]) + length(system.order_queues[2]))
            finished_entities_count += 1  # Increment count for each finished entity
            total_entities = length(system.order_queues[1]) + length(system.order_queues[2])
            total_entities += count(o -> o != nothing, system.in_service)
            push!(n_entities_in_system_over_time, total_entities)
        end
    end
    return (n_entities_in_system_over_time,interarrival_times,queue_lengths_over_time,queue1_wait_times, queue2_wait_times, total_times_in_system, copy(system.machine_idle_times), finished_entities_count)
end

# Run simulations with different seeds
n_runs = 100
all_runs_n_entities_in_system = []
all_queue1_wait_times = []
all_queue2_wait_times = []
all_total_times_in_system = []
all_machine_idle_times = [Float64[] for _ in 1:n_servers]
finished_entities_counts = []
all_queue_lengths = []
all_interarrival_times = []

for seed in 1:n_runs
(n_entities_in_system,interarrival_times,queue_lengths,queue1_wait_times, queue2_wait_times, total_times_in_system, machine_idle_times, finished_count) = run_single_simulation(seed)
    append!(all_queue1_wait_times, queue1_wait_times)
    append!(all_queue2_wait_times, queue2_wait_times)
    append!(all_total_times_in_system, total_times_in_system)
    push!(all_interarrival_times, interarrival_times)
    push!(all_runs_n_entities_in_system, n_entities_in_system)
    for i in 1:n_servers
        push!(all_machine_idle_times[i], machine_idle_times[i])
        push!(finished_entities_counts, finished_count)
        push!(all_queue_lengths, queue_lengths)
    end
end

# Calculate averages and confidence intervals
avg_queue1_wait_time, ci_queue1_wait_time = mean(all_queue1_wait_times), confidence_interval(all_queue1_wait_times)
avg_queue2_wait_time, ci_queue2_wait_time = mean(all_queue2_wait_times), confidence_interval(all_queue2_wait_times)
avg_total_time_in_system, ci_total_time_in_system = mean(all_total_times_in_system), confidence_interval(all_total_times_in_system)
avg_machine_idle_time1, ci_machine_idle_time1 = mean(all_machine_idle_times[1]), confidence_interval(all_machine_idle_times[1])
avg_machine_idle_time2, ci_machine_idle_time2 = mean(all_machine_idle_times[2]), confidence_interval(all_machine_idle_times[2])
avg_finished_entities = mean(finished_entities_counts)

# Write averages, confidence intervals, and parameters to a new CSV file
# Write averages and confidence intervals to a new CSV file
open("simulation_summary_with_confidence_intervals.csv", "w") do file
    println(file, "Statistic,Mean,Lower CI,Upper CI")
    println(file, "Queue 1 Wait Time (min),$avg_queue1_wait_time,$(ci_queue1_wait_time[1]),$(ci_queue1_wait_time[2])")
    println(file, "Queue 2 Wait Time (min),$avg_queue2_wait_time,$(ci_queue2_wait_time[1]),$(ci_queue2_wait_time[2])")
    println(file, "Order,Total Time in System (min),$avg_total_time_in_system,$(ci_total_time_in_system[1]),$(ci_total_time_in_system[2])")
    println(file, "Machine 1 Idle Time (min),$avg_machine_idle_time1,$(ci_machine_idle_time1[1]),$(ci_machine_idle_time1[2])")
    println(file, "Machine 2 Idle Time (min),$avg_machine_idle_time2,$(ci_machine_idle_time2[1]),$(ci_machine_idle_time2[2])")
    println(file, "Number of finished Orders, $avg_finished_entities")
end

# Determine the maximum length of queue lengths array
max_length = maximum(length.(all_queue_lengths))

# Open a CSV file for writing
open("queue_lengths_over_time.csv", "w") do file
    # Write headers
    println(file, join(["Run_$seed" for seed in 1:n_runs], ","))
    
    # Write data
    for i in 1:max_length
        line = [get(all_queue_lengths[j], i, "") for j in 1:n_runs]
        println(file, join(line, ","))
    end
end

function write_to_csv(filename, data)
    open(filename, "w") do file
        max_length = maximum(length.(data))
        for i in 1:max_length
            line = [i <= length(run_data) ? run_data[i] : "" for run_data in data]
            println(file, join(line, ","))
        end
    end
end

write_to_csv("interarrival_times.csv", all_interarrival_times)

using CSV, DataFrames

# Function to pad shorter arrays with missing values
function pad_to_length(arr, len)
    append!(copy(arr), fill(missing, len - length(arr)))
end

# Transpose the data so each run is a column
data_length = maximum(length.(all_runs_n_entities_in_system))
padded_data = [pad_to_length(run_data, data_length) for run_data in all_runs_n_entities_in_system]
df = DataFrame(padded_data, :auto)

# Save to CSV
CSV.write("n_entities_in_system_over_time.csv", df)



0.0: 
0.0: 
25.706982392564758: 
40.49940643026974: 
51.04011165683839: 
90.15604994561741: 
106.25105787025089: 
115.83068465204056: 
136.10162246192667: 
137.5891044791712: 
275.4938918922617: 
289.7200926145116: 
296.8140950771739: 
310.15531122254544: 
319.39306494256675: 
320.30656976174635: 
425.1326001462728: 
454.1271598929508: 
481.3683244468356: 
501.3772135369817: 
501.9124724548472: 
510.32197471047215: 
537.6206844575729: 
539.13307408157: 
559.9627743759228: 
565.4495402052388: 
567.6796778492338: 
583.6470830765768: 
590.6095781324785: 
619.2379526409328: 
658.7944255712229: 
669.7848067561172: 
680.3174918900665: 
688.7362988565766: 
707.5894804210753: 
714.1268070994796: 
742.706837036477: 
789.1352476446785: 
806.1502067106811: 
811.4024568769238: 
811.4556510738736: 
821.2508865426014: 
838.2506040776519: 
868.0981292345147: 
872.10462816871: 
885.6590721239045: 
892.620263093669: 
893.5735790323487: 
918.5919808204844: 
920.5033490288278: 
962.4993973287952: 
976.55

"n_entities_in_system_over_time.csv"